# Mass Transports

Transport diagnostics for flow through major straits.

## Theory

Formally, mass transports are given by
$$T_x = \rho u  $$
$$T_y = \rho v  $$

Mass transports are diagnostics that are calculated online by the model:

| variable|long name|units|dimensions|
| --- |  --- | --- | --- |
| `tx_trans` |T-cell i-mass transport|Sv|(time,st_ocean,xu_ocean,yt_ocean)|
| `ty_trans` |T-cell j-mass transport|Sv|(time,st_ocean,xt_ocean,yu_ocean)|


## Calculation

In [ ]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt

Use default database for this calculation

In [ ]:
session = cc.database.create_session()

This dictionary defines a few key choke points that you might be interested in. For the purposes of demonstration we are just using Drake Passage in this example.

In [ ]:
straights = { 'DrakePassage': [-69.9, -69.9, -71.6, -51.0],
              'Lombok':       [-244.6+360, -243.9+360, -8.6, -8.6],
              'Ombai' :       [-235.0+360, -235.0+360, -9.2, -8.1],
              'Timor' :       [-235.9+360, -235.9+360, -11.9, -9.9],
              'Bering' :      [-172, -167, 65.8, 65.8],
              'Denmark' :     [-42, -22, 65.8, 65.8],
            }

Here we define a function so that you can specify a given strait, and rely on the function to extract the transport through that strait.

In [ ]:
def calc_transport(expt, strait):
    """
    Calculate barotropic transport across a given 
     line of latitude or longitude.
     
    Designed for flow through straits.
    """
    
    print('Calculating {}:{} transport'.format(expt, strait))
    
    xmin, xmax, ymin, ymax = straits[strait]
    
    if xmin == xmax:                
        tx_trans = cc.querying.getvar(expt,'tx_trans',session)
        
        transport = tx_trans.sel(xu_ocean=xmin, method='nearest')\
                            .sel(yt_ocean=slice(ymin, ymax))\
                            .sum('st_ocean').sum('yt_ocean')/1e6/1036

    elif ymin == ymax:
        ty_trans = cc.querying.getvar(expt,'tx_trans',session)
        transport = ty_trans.sel(yu_ocean=ymin, method='nearest')\
                            .sel(xt_ocean=slice(xmin, xmax))\
                            .sum('st_ocean').sum('xt_ocean')/1e6/1036
    else:
        raise ValueError('Transports are computed only along lines of either constant latitude or longitude')
    
    transport = transport.compute()
       
    return transport

Now, for a given experiment, calculate the total (barotropic) transport through Drake Passage:

In [ ]:
%%time
expt = '025deg_jra55v13_ryf9091_gmredi6'
transport = calc_transport(expt, 'DrakePassage')

Plot the result

In [ ]:
transport.plot(linestyle='-')
plt.title('Drake Passage Transport')
plt.xlabel('Year')
plt.ylabel('Transport (Sv)')